In [25]:
import requests
import pandas as pd
import config
import enum
import os
import json
import numpy as np

In [2]:
def get_match_details(match_id):
    root_url = "https://api.steampowered.com"
    command_type = "/IDOTA2Match_570"
    command = "/GetMatchDetails/v1"

    url = root_url + command_type + command
    api_key = config.STEAM_API_KEY
    query_params = {
        "key": api_key,
        "partner": 0,
        "match_id": match_id,
        "include_persona_names": 1,
    }

    res = requests.get(url, query_params)

    if res.ok:
        data = res.json()
        if not "error" in data["result"].keys():

            # with open('match.json', 'w') as f:
            #     json.dump(data, f, indent=4)
            #     f.close()

            return data

    return None

### Get player's matches by his steam_id by using Dota Web API

In [3]:
def get_player_matches(account_id, game_mode, match_requested):
    root_url = "https://api.steampowered.com"
    command_type = "/IDOTA2Match_570"
    command = "/GetMatchHistory/v1"

    url = root_url + command_type + command
    api_key = config.STEAM_API_KEY
    query_params = {
        "key": api_key,
        "partner": 0,
        "account_id": account_id,
        "matches_requested": match_requested,
        "game_mode": game_mode,
    }

    res = requests.get(url, query_params)

    if res.ok:
        data = res.json()

        # with open(os.getcwd() + '/data/playerStatsInMatches.json', 'w') as f:
        #     json.dump(data, f, indent=4)
        #     f.close()

        return data

    return None

In [4]:
def get_player_match_stats(match_id, account_id):
    match = get_match_details(match_id)
    player_stats = {
        "persona": "",
        "team_number": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "last_hits": 254,
        "denies": 0,
        "gold_per_min": 0,
        "xp_per_min": 0,
        "level": 0,
        "net_worth": 0,
        "hero_damage": 0,
        "tower_damage": 0,
        "hero_healing": 0,
        "gold_spent": 0,
        "scaled_hero_damage": 0,
        "scaled_tower_damage": 0,
        "scaled_hero_healing": 0,
    }

    if match is not None:
        for player in match["result"]["players"]:
            try:
                if player["account_id"] == account_id:
                    for stat in player_stats.keys():
                        player_stats[stat] = player[stat]
            except Exception as e:
                continue

        #!print(player_stats)

        player_stats["win"] = (
            1
            if (
                not player_stats["team_number"]
                and match["result"]["radiant_win"]
                or player_stats["team_number"]
                and not match["result"]["radiant_win"]
            )
            else 0
        )
        # print('match:', match['result']['match_id'], 'team_number', player_stats['team_number'], 'radiant_win:',
        #       match['result']['radiant_win'], 'win:', player_stats['win'])

        return player_stats

    return None

### Get one player statistics

In [5]:
def process_player_data(account_id, game_mode, match_requested):
    data = get_player_matches(account_id, game_mode, match_requested)
    name = ""
    middle_stats = {
        "win": 0,
        "team_number": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "last_hits": 254,
        "denies": 0,
        "gold_per_min": 0,
        "xp_per_min": 0,
        "level": 0,
        "net_worth": 0,
        "hero_damage": 0,
        "tower_damage": 0,
        "hero_healing": 0,
        "gold_spent": 0,
        "scaled_hero_damage": 0,
        "scaled_tower_damage": 0,
        "scaled_hero_healing": 0,
    }

    if data is not None and "matches" in data["result"].keys():
        for match in data["result"]["matches"]:
            #!print(match['match_id'])
            match_stats = get_player_match_stats(match["match_id"], account_id)

            if match is not None:
                name = match_stats["persona"]
                del match_stats["persona"]
                for stat in match_stats.keys():
                    middle_stats[stat] += match_stats[stat]

        for stat in middle_stats.keys():
            middle_stats[stat] /= len(data["result"]["matches"])

        middle_stats["team_number"] = 1 - middle_stats["team_number"]
        middle_stats["matches"] = len(data["result"]["matches"])
        middle_stats["radiant"] = middle_stats["team_number"]
        middle_stats["name"] = name
        middle_stats["account_id"] = account_id

        del middle_stats["team_number"]

        return middle_stats

    return None

In [6]:
class GameMode(enum.Enum):
    NO_MODE = 0
    ALL_PICK = 1
    CAP_MODE = 2
    RANDOM_DRAFT = 3
    SINGLE_DRAFT = 4

####   Get player's with steam_id = 178749178 info about last 20 matches in all pick

In [8]:
data = process_player_data(178749178, GameMode.ALL_PICK, 20)
data

{'win': 0.6,
 'kills': 7.25,
 'deaths': 6.4,
 'assists': 9.9,
 'last_hits': 173.9,
 'denies': 5.25,
 'gold_per_min': 1170.35,
 'xp_per_min': 1858.0,
 'level': 26.15,
 'net_worth': 29869.35,
 'hero_damage': 24293.3,
 'tower_damage': 6454.55,
 'hero_healing': 176.95,
 'gold_spent': 25979.5,
 'scaled_hero_damage': 11486.25,
 'scaled_tower_damage': 3093.0,
 'scaled_hero_healing': 88.1,
 'matches': 20,
 'radiant': 0.5,
 'name': 'скуфлер',
 'account_id': 178749178}

In [7]:
def json2DF(json_data):
    return pd.json_normalize(json_data)

### Get stats top 10 world players

In [9]:
def get_players_df_stats(player_ids : list[int], matches_count : int, game_mode : GameMode) -> pd.DataFrame:
    json_data = []

    for id in player_ids:
        json_data.append(process_player_data(account_id=id, game_mode=game_mode, match_requested=matches_count))

    return json2DF(json_data)

In [15]:
df_players = get_players_df_stats(
    [
        171262902,
        93618577,
        321580662,
        73401082,
        898455820,
        183719386,
        56351509,
        480412663,
        1044002267,
        302214028,
    ],
    20,
    GameMode.ALL_PICK,
)

df_players

,win,kills,deaths,assists,last_hits,denies,gold_per_min,xp_per_min,level,net_worth,...,tower_damage,hero_healing,gold_spent,scaled_hero_damage,scaled_tower_damage,scaled_hero_healing,matches,radiant,name,account_id
0,0.65,10.50,3.00,9.90,308.70,15.10,813.85,967.05,21.65,22991.60,...,10331.80,259.50,21196.00,15633.55,4664.85,109.30,20,0.55,1,171262902
1,0.75,10.00,5.05,12.10,263.10,12.30,680.85,821.20,20.80,20533.30,...,4479.50,112.40,20012.75,17902.45,2101.55,60.45,20,0.40,ammar is my goat,93618577
2,0.55,12.25,4.80,10.05,380.75,8.65,715.25,898.85,23.60,26583.40,...,8953.70,1444.05,26882.00,17253.95,3964.75,375.80,20,0.20,yao,321580662
3,0.50,2.30,5.95,16.40,78.00,2.95,335.90,434.25,15.80,9961.60,...,2631.10,6270.65,10261.25,8470.00,1202.90,1904.60,20,0.60,Dukalis,73401082
4,0.60,8.60,4.30,10.35,316.85,13.75,634.50,740.75,20.85,20524.20,...,6634.70,0.00,20125.25,16749.10,2982.40,0.00,20,0.55,Буська,898455820
5,0.60,9.40,4.60,11.25,308.30,14.70,630.80,761.45,21.35,21940.00,...,4814.40,0.00,22800.50,16155.35,2403.65,0.00,20,0.50,Suma1L-,183719386
6,0.50,5.55,4.40,16.10,262.55,11.90,552.70,640.45,19.85,18565.45,...,2432.80,2089.20,17869.50,11922.95,1074.10,752.60,20,0.45,Offlane Bobi,56351509
7,0.45,9.20,4.55,11.15,343.95,12.00,645.50,770.30,21.65,21759.60,...,3288.60,832.20,21593.00,18625.20,1485.40,340.25,20,0.55,yuuuuuuuuuuuuuuuuuuuuuuuuuuuu,480412663
8,0.65,9.50,2.95,8.85,424.05,12.40,748.00,873.40,22.30,25541.75,...,9943.65,47.40,25379.50,14865.85,4490.30,21.95,20,0.50,4,1044002267
9,0.60,6.20,3.55,12.05,241.05,11.30,555.05,653.25,19.35,18168.75,...,2062.50,908.05,16901.00,10543.65,909.85,358.85,20,0.75,cao,302214028


,Unnamed: 0,win,kills,deaths,assists,last_hits,denies,gold_per_min,xp_per_min,level,...,tower_damage,hero_healing,gold_spent,scaled_hero_damage,scaled_tower_damage,scaled_hero_healing,matches,radiant,name,account_id
0,0,0.65,10.50,3.00,9.90,308.70,15.10,813.85,967.05,21.65,...,10331.80,259.50,21196.00,15633.55,4664.85,109.30,20,0.55,1,171262902
1,1,0.75,10.00,5.05,12.10,263.10,12.30,680.85,821.20,20.80,...,4479.50,112.40,20012.75,17902.45,2101.55,60.45,20,0.40,ammar is my goat,93618577
2,2,0.55,12.25,4.80,10.05,380.75,8.65,715.25,898.85,23.60,...,8953.70,1444.05,26882.00,17253.95,3964.75,375.80,20,0.20,yao,321580662
3,3,0.50,2.30,5.95,16.40,78.00,2.95,335.90,434.25,15.80,...,2631.10,6270.65,10261.25,8470.00,1202.90,1904.60,20,0.60,Dukalis,73401082
4,4,0.60,8.60,4.30,10.35,316.85,13.75,634.50,740.75,20.85,...,6634.70,0.00,20125.25,16749.10,2982.40,0.00,20,0.55,Буська,898455820
5,5,0.60,9.40,4.60,11.25,308.30,14.70,630.80,761.45,21.35,...,4814.40,0.00,22800.50,16155.35,2403.65,0.00,20,0.50,Suma1L-,183719386
6,6,0.50,5.55,4.40,16.10,262.55,11.90,552.70,640.45,19.85,...,2432.80,2089.20,17869.50,11922.95,1074.10,752.60,20,0.45,Offlane Bobi,56351509
7,7,0.45,9.20,4.55,11.15,343.95,12.00,645.50,770.30,21.65,...,3288.60,832.20,21593.00,18625.20,1485.40,340.25,20,0.55,yuuuuuuuuuuuuuuuuuuuuuuuuuuuu,480412663
8,8,0.65,9.50,2.95,8.85,424.05,12.40,748.00,873.40,22.30,...,9943.65,47.40,25379.50,14865.85,4490.30,21.95,20,0.50,4,1044002267
9,9,0.60,6.20,3.55,12.05,241.05,11.30,555.05,653.25,19.35,...,2062.50,908.05,16901.00,10543.65,909.85,358.85,20,0.75,cao,302214028


### Analyse TI7 Finals (Team Liquid)
> 1. Want to know how team liquid played (their stats)
> 2. What heroes they picked on average
> 3. What items on they picked on average


In [18]:
ti7_finals = [3372622939, 3372676225, 3372726385]
matches_info = [get_match_details(match) for match in ti7_finals]

##### Their picks

In [21]:
ind = 0
for match in matches_info:
    if match is not None:
        with open(os.getcwd() + '/data/match' + str(ind) + '.json', encoding='utf-8', mode='w') as f:
            json.dump(match, f, indent=4, ensure_ascii=False)
        ind += 1

In [32]:
team_number = [1, 0, 1] #   Team Liquid
team_picks = np.ndarray(shape=3)

#   Try to analyse only first match

for ind, match in enumerate(matches_info[:1]):
    df_players = pd.json_normalize(match["result"]["players"])
    print(df_players[df_players["team_number"] == team_number[ind]])

   account_id  player_slot  team_number  team_slot  hero_id  item_0  item_1  \
5   105248644          128            1          0       95     172     212   
6    34505203          129            1          1       53      50     116   
7    82262664          130            1          2       31       0      43   
8   101356886          131            1          3       28     214      34   
9    72312627          132            1          4       54      50       0   

   item_2  item_3  item_4  ...  last_hits  denies  gold_per_min  xp_per_min  \
5     174     116      50  ...        203      20           546         521   
6      36     152      46  ...        179      14           538         433   
7     214     254      36  ...         33       1           296         443   
8      46       0       1  ...         18       1           293         306   
9      36     151     168  ...        137       9           562         587   

   level  net_worth  aghanims_scepter  aghanims_sh